In [ ]:
%pip install openai-agents
%pip install openai-agents[litellm]

In [86]:
import os
os.environ["GEMINI_API_KEY"] = ""
print(os.getenv("GEMINI_API_KEY")) 

In [10]:
from agents import Agent ,Runner
import asyncio

In [12]:
# def main():
#     agent = Agent (name= "Tester" , instructions="Answer the question ", model="litellm/gemini/gemini-1.5-flash")

#     result = await Runner.run(agent , "write a motivational quote of work")
#     print(result.final_output)

In [19]:
import asyncio
from agents import Agent, Runner

# creating an agent 
agent = Agent(
    name = "Gemini agent",
    instructions = "You are a helpfull assistant to guide the user query ",
    model = "litellm/gemini/gemini-1.5-flash"
)

In [20]:
async def main():
    result = await Runner.run(agent,input="Can you tell me what is the capital of Pakistan?")
    print("Agent : " , result.final_output)

In [21]:
await main()

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\typing.py:407: RuntimeWarning: coroutine 'main' was never awaited
  def _eval_type(t, globalns, localns, type_params=None, *, recursive_guard=frozenset()):


Agent :  The capital of Pakistan is **Islamabad**.



# Basic configuration

In [ ]:
from agents import Agent , Runner

agent = Agent(
    name="Helpfull Assistant",
    instructions = "You are the helpful assistant",
    model = "litellm/gemini/gemini-1.5-flash"
)

# result = Runner.run_sync(agent , input="What is the capital of France?")
# print(result.final_output)

# Context

In [2]:
from agents import Agent , Runner , function_tool
import os 
import asyncio

In [85]:
os.environ["GEMINI_API_KEY"] = ""
print(os.getenv("GEMINI_API_KEY")) 

In [27]:
# Creating a Context for shopping assistant
from dataclasses import dataclass


@dataclass
class ShoppingAssistantContext:
    name: str 
    age: int
    phoneno: int
    email: str

In [28]:
comtext = ShoppingAssistantContext(
    name = "Tayyab ELlahi",
    age = 20,
    phoneno= 33344455566,
    email= "tayyabeh1807@gmail.com"
)

In [ ]:
# dynamic instruction
def dynamic_instruction(comtext , agent):
    return f"You are a helpful shopping assistant. The user's name is {comtext.name}. Greet the user by their name and ask how you can help them."

In [ ]:
# Creating an agent
agent = Agent(
    name = "Shopping Assistant Agent",
    instructions = dynamic_instruction,
    model = "litellm/gemini/gemini-2.5-flash"
)

In [34]:
# main 
async def main():
    result = await Runner.run(agent , input = "Plz help me in shopping..i want obuy a phone iphone 15",context = comtext)
    print(result.final_output)

if __name__ == "__main__":
    await(main())


Hi Tayyab Ellahi,

It's great to hear from you!  I'd be happy to help you with your shopping for an iPhone 15. To best assist you, could you tell me a little more about what you're looking for?  For example:

* **Which model of the iPhone 15 are you interested in?** (iPhone 15, iPhone 15 Plus, iPhone 15 Pro, iPhone 15 Pro Max)
* **What's your budget?**
* **Where are you planning to buy it from?** (e.g., Apple Store, carrier, online retailer)
* **Are there any specific features that are important to you?** (e.g., camera quality, storage capacity, battery life)

The more information you give me, the better I can assist you in finding the perfect iPhone 15 for your needs.



# Output Types

In [38]:
from agents import Agent, Runner
from pydantic import BaseModel, EmailStr
import asyncio
import os


In [41]:
class ContactInfo(BaseModel):
    name: str
    phoneno : str
    email: EmailStr
    

In [ ]:
agent = Agent(
    name = "Extract Infor Agent", 
    instructions = "You are a helpfull assistant that can extract user infor from user input",
    model = "litellm/gemini/gemini-2.5-flash",
    output_type = ContactInfo    
)

In [43]:
user_input = 'hello i am tayyab ellahi and i m a passionate ai engineer and you can contact me through email tayyab@gmail.com and also contact me from phone 9233345434124'

async def main():
    result = await Runner.run(agent,user_input)
    print(result.final_output)

if __name__ == '__main__':
    await main()

name='tayyab ellahi' phoneno='9233345434124' email='tayyab@gmail.com'


# HandsOff

GUARDRAIL IMPLEMENTED

In [247]:
from agents import Agent , Runner, handoff, RunContextWrapper, HandoffInputData, GuardrailFunctionOutput , InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered, input_guardrail,  output_guardrail
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from langchain_google_genai import GoogleGenerativeAI
from pydantic import BaseModel, EmailStr
import asyncio
import os


In [301]:
os.environ["GEMINI_API_KEY"] = ""
print(os.getenv("GEMINI_API_KEY")) 

In [249]:
# import getpass

# if not os.environ.get("GOOGLE_API_KEY"):
#   os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [250]:
# Response output agent 
# -------- Pydantic Schemas --------
class GuardrailCheckOutput(BaseModel):
    is_valid_output: bool
    reasoning: str

In [251]:
class FinalAnswer(BaseModel):
    source_agent: str
    user_question: str
    final_response: str

In [252]:
class TechnicalProblem(BaseModel):
    product_id : str
    description : str

In [253]:
class BillingProblem(BaseModel):
    invoice_no : str
    problem_type : str

In [270]:
guardrail_agent = Agent(
    name = "Guardrial Check",
    instructions= '''
    You are a guardrail. Your job is to check if the user's query is related to **customer support**.

    Customer support topics include:
    - Billing problems
    - Technical problems

    If the query is related to customer support → set `is_valid_output=true`.
    Otherwise (like math questions, chit-chat, unrelated queries) → set `is_valid_output=false`.

    Always respond in valid JSON with fields:
    {
      "is_valid_output": true/false,
      "reasoning": "your short explanation"
    }
    ''',
    model = "litellm/gemini/gemini-2.5-flash",
    output_type = GuardrailCheckOutput
)

In [255]:
# -------- Output Guardrail Agent --------
final_answer_guardrail_agent = Agent(
    name="FinalAnswerGuardrail",
    instructions=f"""
    You are a strict guardrail. Check if the given output is a valid JSON that matches the FinalAnswer format.
    The format requires exactly these three string fields: {{ "source_agent": "...", "user_question": "...", "final_response": "..." }}
    If it is valid, set is_valid_output=true. Otherwise, set is_valid_output=false.
    """,
    model="litellm/gemini/gemini-2.5-flash",
    output_type=GuardrailCheckOutput
)

In [269]:
@input_guardrail
async def customer_guardrail(ctx: RunContextWrapper , agent :Agent, input:str) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input , context = ctx.context)
    return GuardrailFunctionOutput(
        output_info = result.final_output,
        tripwire_triggered = not  result.final_output.is_valid_output
    )

In [271]:
@output_guardrail
async def final_answer_guardrail(ctx: RunContextWrapper, agent: Agent, output: FinalAnswer) -> GuardrailFunctionOutput:
    
    # result = await Runner.run(final_answer_guardrail_agent, output, context=ctx.context)
    output_dict = output.model_dump() if hasattr(output, 'model_dump') else output.dict()
    result = await Runner.run(final_answer_guardrail_agent, json.dumps(output_dict), context=ctx.context)
    
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=not result.final_output.is_valid_output
    )

In [272]:
# function input filter function 
# def billing_input_filter(input:HandoffInputData):   
# #     if len(problem.invoice_no) != 6 :
# #         raise ValueError("your invoice must be 6 digit number")
#     print(input.input_history)
#     # Nested dictionary aur list se 'arguments' ki value tak pohunchna
#     arguments_value = input['HandoffInputData']['new_items'][0]['raw_item']['arguments']
    
#     # Value ko print karna
#     print(arguments_value)
#     return input


In [273]:
import json
# Agents SDK ke zaroori imports yahan honge (farz kar lein)
# from agents.runtime import HandoffInputData 

def billing_input_filter(input: HandoffInputData):
    """
    Yeh filter HandoffInputData object se invoice number nikalta hai
    aur uski length ko validate karta hai.
    """
    try:
        # 1. Object ke attributes ko dot (.) se access karein
        # 'new_items' ek list/tuple hai, isliye pehla item [0] se lein.
        # 'raw_item' ek aur object hai, jiske andar 'arguments' hai.
        arguments_string = input.new_items[0].raw_item.arguments
        
        # 2. 'arguments' ek JSON string hai. Ise Python dictionary mein convert karein.
        arguments_dict = json.loads(arguments_string)
        
        # 3. Dictionary se 'invoice_no' nikalein.
        invoice_no = arguments_dict['invoice_no']

        print(f"Invoice number mila: {invoice_no}") # Debugging ke liye

        # 4. Ab aap apna validation logic chala sakte hain.
        if len(invoice_no) != 6:
            # Agar validation fail ho, to error raise karein. SDK isko handle karega.
            raise ValueError("Aapka invoice 6 digit ka hona chahiye.")
        
        print("Invoice number validation pass ho gaya!")

    except (IndexError, KeyError, AttributeError) as e:
        # Agar invoice number nahi milta to error raise karein
        raise ValueError(f"Invoice number input data mein nahi mil saka. Error: {e}")

    # Agar sab kuch theek hai, to original input ko wapas return karein.
    return input

In [274]:
def on_handoff_callback_technical(ctx : RunContextWrapper , input: TechnicalProblem):
    print( "Yes the technical Agent is called!!")
    print(f"Data being passed: {input}")

In [ ]:
# --- TECHNICAL AGENT (FINAL OUTPUT GENERATOR) ---
technical_support_agent = Agent(
    name="Technical Support Agent",
    instructions=f'''
    {RECOMMENDED_PROMPT_PREFIX} 
    You are a helpful Technical Support assistant. The user has been transferred to you.
    Your final task is to provide a helpful response and format it as a JSON object of type FinalAnswer.
    - Set source_agent to your name: "Technical Support Agent".
    - Set user_question to the user's original problem.
    - Set final_response to your helpful reply.
    ''',
    model="litellm/gemini/gemini-2.5-flash",
    output_type=FinalAnswer,                # <-- Yahan laga!
    output_guardrails=[final_answer_guardrail] # <-- Yahan laga!
)

In [284]:
tech_support_handoff = handoff(
    agent = technical_support_agent,
    tool_name_override = 'Transfer_to_Technical_support',
    tool_description_override = "Transfers a request to the technical support team.",
    on_handoff = on_handoff_callback_technical,
    input_type = TechnicalProblem
)

In [285]:
# --- BILLING AGENT (FINAL OUTPUT GENERATOR) ---
billing_agent = Agent(
    name="Billing Agent",
    instructions=f'''
    {RECOMMENDED_PROMPT_PREFIX} 
    You are a helpful billing assistant. The user has been transferred to you.
    Your final task is to provide a helpful response and format it as a JSON object of type FinalAnswer.
    - Set source_agent to your name: "Billing Agent".
    - Set user_question to the user's original problem.
    - Set final_response to your helpful reply.
    ''',
    model="litellm/gemini/gemini-2.5-flash",
    output_type=FinalAnswer,                # <-- Yahan bhi laga!
    output_guardrails=[final_answer_guardrail] # <-- Yahan bhi laga!
)

In [286]:
def on_handoff_callback_billing(ctx : RunContextWrapper , input_data: BillingProblem):
    print(f"Yes the billing Agent is called!! ")

In [287]:
billing_handoff = handoff(
    agent = billing_agent,    
    on_handoff = on_handoff_callback_billing,
    input_type = BillingProblem,
    input_filter = billing_input_filter
)

In [288]:
triage_agent= Agent( 
    name = "Triage Agent", 
    instructions=f""" 
    {RECOMMENDED_PROMPT_PREFIX} 
   You are a master router. Your job is to analyze the user's query and call the appropriate tool.
    
    - For technical problems (device issues, hardware/software problems, product malfunctions): call Transfer_to_Technical_Support
    - For billing problems (invoice issues, payment problems, billing questions): call Transfer_to_Billing_Support
    
    Always analyze the query carefully and route to the most appropriate specialist.
    Never answer directly - always route to the appropriate specialist team.
    
    Extract the relevant information:
    - For technical issues: extract product_id and description
    - For billing issues: extract invoice_no and problem_type
   

    """, 
    model = "litellm/gemini/gemini-2.5-flash", 
    handoffs=[tech_support_handoff,billing_handoff], 
    input_guardrails=[customer_guardrail]       
    )

In [289]:
async def main():
    try:
        result = await Runner.run(
            triage_agent, 
            input = """
        Hello, I have a problem with my phone, product ID 'IPH-123'. 
        Its charger is not working fine at all.
            """
            )
        print(result)
    except InputGuardrailTripwireTriggered :
        print("Bro this is just for customer service only 🚫")
    except OutputGuardrailTripwireTriggered:
        print("🚫 Output format invalid (math or non-customer support)")
        


In [290]:
if __name__ == '__main__':
    await main()
    

Yes the technical Agent is called!!
Data being passed: product_id='IPH-123' description='Charger is not working fine at all.'


AttributeError: 'str' object has no attribute 'dict'

In [92]:
'''
Hello, i need help i want to tell u i have problem in my phone
        its charger is not working fine

'''

'\nHello, i need help i want to tell u i have problem in my phone\n        its charger is not working fine\n\n'

In [246]:
from agents import Agent, Runner, handoff, RunContextWrapper, HandoffInputData, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered, input_guardrail, output_guardrail
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from langchain_google_genai import GoogleGenerativeAI
from pydantic import BaseModel, EmailStr
import asyncio
import os
import json

# -------- Pydantic Schemas --------
class GuardrailCheckOutput(BaseModel):
    is_valid_output: bool
    reasoning: str

class FinalAnswer(BaseModel):
    source_agent: str
    user_question: str
    final_response: str

class TechnicalProblem(BaseModel):
    product_id: str
    description: str

class BillingProblem(BaseModel):
    invoice_no: str
    problem_type: str

# -------- Guardrail Agents --------
guardrail_agent = Agent(
    name="Guardrail Check",
    instructions='''
    You are a guardrail. Your job is to check if the user's query is related to **customer support**.

    Customer support topics include:
    - Billing problems
    - Technical problems
    - Product issues
    - Support requests

    If the query is related to customer support → set `is_valid_output=true`.
    Otherwise (like math questions, chit-chat, unrelated queries) → set `is_valid_output=false`.

    Always respond in valid JSON with fields:
    {
      "is_valid_output": true/false,
      "reasoning": "your short explanation"
    }
    ''',
    model="litellm/gemini/gemini-2.5-flash",
    output_type=GuardrailCheckOutput
)

final_answer_guardrail_agent = Agent(
    name="FinalAnswerGuardrail",
    instructions=f"""
    You are a strict guardrail. Check if the given output is a valid JSON that matches the FinalAnswer format.
    The format requires exactly these three string fields: {{ "source_agent": "...", "user_question": "...", "final_response": "..." }}
    If it is valid, set is_valid_output=true. Otherwise, set is_valid_output=false.
    """,
    model="litellm/gemini/gemini-2.5-flash",
    output_type=GuardrailCheckOutput
)

# -------- Guardrail Functions --------
@input_guardrail
async def customer_guardrail(ctx: RunContextWrapper, agent: Agent, input: str) -> GuardrailFunctionOutput:
    try:
        result = await Runner.run(guardrail_agent, input, context=ctx.context)
        return GuardrailFunctionOutput(
            output_info=result.final_output,
            tripwire_triggered=not result.final_output.is_valid_output
        )
    except Exception as e:
        print(f"Guardrail error: {e}")
        # Default to allowing customer support queries if guardrail fails
        return GuardrailFunctionOutput(
            output_info=GuardrailCheckOutput(is_valid_output=True, reasoning="Guardrail failed, defaulting to allow"),
            tripwire_triggered=False
        )

@output_guardrail
async def final_answer_guardrail(ctx: RunContextWrapper, agent: Agent, output: FinalAnswer) -> GuardrailFunctionOutput:
    try:
        # Convert FinalAnswer to dict for validation
        output_dict = output.model_dump() if hasattr(output, 'model_dump') else output.dict()
        result = await Runner.run(final_answer_guardrail_agent, json.dumps(output_dict), context=ctx.context)
        return GuardrailFunctionOutput(
            output_info=result.final_output,
            tripwire_triggered=not result.final_output.is_valid_output
        )
    except Exception as e:
        print(f"Output guardrail error: {e}")
        # Default to allowing valid FinalAnswer objects
        return GuardrailFunctionOutput(
            output_info=GuardrailCheckOutput(is_valid_output=True, reasoning="Output guardrail failed, defaulting to allow"),
            tripwire_triggered=False
        )

# -------- Input Filter Functions --------
def billing_input_filter(input_data: HandoffInputData):
    """
    Filter to validate billing input data
    """
    try:
        # Access the arguments from HandoffInputData
        if hasattr(input_data, 'new_items') and len(input_data.new_items) > 0:
            # Get the first item's arguments
            arguments_string = input_data.new_items[0].raw_item.arguments
            arguments_dict = json.loads(arguments_string)
            invoice_no = arguments_dict.get('invoice_no', '')
            
            print(f"Invoice number received: {invoice_no}")
            
            # Validate invoice number length
            if len(invoice_no) != 6:
                raise ValueError("Invoice number must be exactly 6 digits long.")
            
            print("Invoice number validation passed!")
            
        return input_data
        
    except (IndexError, KeyError, AttributeError, json.JSONDecodeError) as e:
        print(f"Input filter error: {e}")
        raise ValueError(f"Invalid invoice data format. Error: {e}")

# -------- Callback Functions --------
def on_handoff_callback_technical(ctx: RunContextWrapper, input_data: TechnicalProblem):
    print("✅ Technical Agent called!")
    print(f"Data being passed: {input_data}")

def on_handoff_callback_billing(ctx: RunContextWrapper, input_data: BillingProblem):
    print("✅ Billing Agent called!")
    print(f"Data being passed: {input_data}")

# -------- Agent Definitions --------
technical_support_agent = Agent(
    name="Technical Support Agent",
    instructions=f'''
    {RECOMMENDED_PROMPT_PREFIX}
    You are a helpful Technical Support assistant. The user has been transferred to you.
    Your task is to provide a helpful response and format it as a JSON object of type FinalAnswer.
    - Set source_agent to your name: "Technical Support Agent"
    - Set user_question to the user's original problem
    - Set final_response to your helpful technical support reply
    
    Provide clear, actionable solutions for technical problems.
    ''',
    model="litellm/gemini/gemini-2.5-flash",
    output_type=FinalAnswer,
    output_guardrails=[final_answer_guardrail]
)

billing_agent = Agent(
    name="Billing Agent",
    instructions=f'''
    {RECOMMENDED_PROMPT_PREFIX}
    You are a helpful billing assistant. The user has been transferred to you.
    Your task is to provide a helpful response and format it as a JSON object of type FinalAnswer.
    - Set source_agent to your name: "Billing Agent"
    - Set user_question to the user's original problem
    - Set final_response to your helpful billing support reply
    
    Help users with billing issues, payment problems, and invoice questions.
    ''',
    model="litellm/gemini/gemini-2.5-flash",
    output_type=FinalAnswer,
    output_guardrails=[final_answer_guardrail]
)

# -------- Handoff Definitions --------
tech_support_handoff = handoff(
    agent=technical_support_agent,
    tool_name_override='Transfer_to_Technical_Support',
    tool_description_override="Transfers a request to the technical support team for hardware/software issues.",
    on_handoff=on_handoff_callback_technical,
    input_type=TechnicalProblem
)

billing_handoff = handoff(
    agent=billing_agent,
    tool_name_override='Transfer_to_Billing_Support',
    tool_description_override="Transfers a request to the billing team for payment and invoice issues.",
    on_handoff=on_handoff_callback_billing,
    input_type=BillingProblem,
    input_filter=billing_input_filter
)

# -------- Triage Agent --------
triage_agent = Agent(
    name="Triage Agent",
    instructions=f'''
    {RECOMMENDED_PROMPT_PREFIX}
    You are a master router. Your job is to analyze the user's query and call the appropriate tool.
    
    - For technical problems (device issues, hardware/software problems, product malfunctions): call Transfer_to_Technical_Support
    - For billing problems (invoice issues, payment problems, billing questions): call Transfer_to_Billing_Support
    
    Always analyze the query carefully and route to the most appropriate specialist.
    Never answer directly - always route to the appropriate specialist team.
    
    Extract the relevant information:
    - For technical issues: extract product_id and description
    - For billing issues: extract invoice_no and problem_type
    ''',
    model="litellm/gemini/gemini-2.5-flash",
    handoffs=[tech_support_handoff, billing_handoff],
    input_guardrails=[customer_guardrail]
)

# -------- Main Function --------
async def main():
    try:
        print("🔄 Starting customer support workflow...")
        
        result = await Runner.run(
            triage_agent,
            input="""
            Hello, I have a problem with my phone, product ID 'IPH-123'. 
            Its charger is not working fine at all.
            """
        )
        
        print("✅ Workflow completed successfully!")
        print("📋 Final Result:")
        print(result)
        
    except InputGuardrailTripwireTriggered as e:
        print("🚫 Input blocked: This service is only for customer support queries.")
        print(f"Details: {e}")
        
    except OutputGuardrailTripwireTriggered as e:
        print("🚫 Output blocked: Invalid response format detected.")
        print(f"Details: {e}")
        
    except Exception as e:
        print(f"❌ Unexpected error occurred: {e}")
        print(f"Error type: {type(e).__name__}")
        import traceback
        traceback.print_exc()

# -------- Execution --------
if __name__ == '__main__':
    await main()

🔄 Starting customer support workflow...
✅ Technical Agent called!
Data being passed: product_id='IPH-123' description='Its charger is not working fine at all.'
✅ Workflow completed successfully!
📋 Final Result:
RunResult:
- Last agent: Agent(name="Technical Support Agent", ...)
- Final output (FinalAnswer):
    {
      "source_agent": "Technical Support Agent",
      "user_question": "Hello, I have a problem with my phone, product ID 'IPH-123'. Its charger is not working fine at all.",
      "final_response": "I'm sorry to hear you're having trouble with your charger for product ID 'IPH-123'. Let's try to troubleshoot this. First, please ensure the charger is securely plugged into both the wall outlet and your phone. If it's still not working, try plugging the charger into a different wall outlet to rule out a power source issue. You could also try using a different cable with the same charging adapter, or a different adapter with the same cable, if you have spares available. This help

# Streaming

In [292]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model = "litellm/gemini/gemini-2.5-flash"
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


if __name__ == "__main__":
    await (main())

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 10
-- Message output:
 Here are 10 jokes for you!

1. Why don't scientists trust atoms? Because they make up everything!
2. What do you call a fake noodle? An impasta!
3. Why did the scarecrow win an award? Because he was outstanding in his field!
4. What do you call a cheese that isn't yours? Nacho cheese!
5. Why don't skeletons fight each other? They don't have the guts!
6. What's orange and sounds like a parrot? A carrot!
7. What do you call a boomerang that won't come back? A stick!
8. Why did the bicycle fall over? Because it was two tired!
9. What did the grape say when it got stepped on? Nothing, it just let out a little wine!
10. How do you organize a space party? You "planet"!
=== Run complete ===


In [31]:
import asyncio
from agents import Agent, Runner,RunConfig

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
    )
    run_config = RunConfig(
        model = "litellm/gemini/gemini-2.5-flash",
        workflow_name = 'Agent workflow',
        trace_id = '1',
        group_id = 'g1'
    )

    result = Runner.run_streamed(agent, input="write a 500 lines essay", run_config = run_config)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)


if __name__ == "__main__":
    await (main())


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



MidStreamFallbackError: litellm.ServiceUnavailableError: litellm.MidStreamFallbackError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - b'{\n  "error": {\n    "code": 429,\n    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",\n    "status": "RESOURCE_EXHAUSTED",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.rpc.QuotaFailure",\n        "violations": [\n          {\n            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",\n            "quotaId": "GenerateRequestsPerDayPerProjectPerModel-FreeTier",\n            "quotaDimensions": {\n              "location": "global",\n              "model": "gemini-2.5-flash"\n            },\n            "quotaValue": "250"\n          }\n        ]\n      },\n      {\n        "@type": "type.googleapis.com/google.rpc.Help",\n      

In [ ]:
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
async def main():
    agent = Agent(
        name = "Assitant",
        instructions= "You are a helpful assistant",
        model= "litellm/gemini/gemini-2.5-flash"
    )
    result = Runner.run_streamed(agent,input="Write a essay on Imran khan of 500 words")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event)
            print(event.type)
            print(event.data.delta, end="", flush=True)
    

    
if __name__ == "__main__":
    await (main())